In [2]:
# Clonar os repositórios
!git clone https://github.com/lyuwenyu/RT-DETR.git
!git clone https://github.com/pedrozamboni/individual_urban_tree_crown_detection.git

Cloning into 'RT-DETR'...
remote: Enumerating objects: 986, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 986 (delta 1), reused 4 (delta 1), pack-reused 978 (from 1)
Receiving objects: 100% (986/986), 620.84 KiB | 4.67 MiB/s, done.
Resolving deltas: 100% (459/459), done.
Cloning into 'individual_urban_tree_crown_detection'...
remote: Enumerating objects: 696, done.
remote: Total 696 (delta 0), reused 0 (delta 0), pack-reused 696 (from 1)
Receiving objects: 100% (696/696), 184.86 MiB | 54.63 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Updating files: 100% (677/677), done.


In [3]:
import os
from os.path import exists, join, basename, splitext

# Alterando para o diretório correto, onde o repositório foi clonado
%cd /content/RT-DETR

# Instalar as dependências, se necessário
!pip install torch==2.0.1
!pip install torchvision==0.15.2
!pip install onnx==1.14.0
!pip install onnxruntime==1.15.1
!pip install pycocotools
!pip install PyYAML
!pip install scipy

# Adicionar o repositório RT-DETR ao sys.path para importar os módulos
import sys
sys.path.append('/content/RT-DETR')


/content/RT-DETR
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [12]:
import os
import shutil
import json
import random
from PIL import Image

# Caminhos das pastas
base_dir = '/content/individual_urban_tree_crown_detection/'
images_dir = os.path.join(base_dir, 'rgb')
annotations_dir = os.path.join(base_dir, 'bbox_txt')
img_list_dir = os.path.join(base_dir, 'img_list/0')

# Novos diretórios
output_dir = '/content/individual_urban_tree_crown_detection/'
output_images_dir = os.path.join(output_dir, 'images')
output_annotations_dir = os.path.join(output_dir, 'annotations')

# Certifique-se de que os diretórios de saída existem
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_annotations_dir, exist_ok=True)

# Função para processar o conjunto (train, val, test)
def process_set(set_name, image_files, annotation_files, verbose=True):
    # Criar diretório para as imagens do conjunto
    set_images_dir = os.path.join(output_images_dir, set_name)
    os.makedirs(set_images_dir, exist_ok=True)

    images = []
    annotations = []
    annotation_id = 1  # ID único para cada anotação

    for image_id, image_file in enumerate(image_files, 1):
        # Copiar a imagem para o diretório correspondente
        src_image_path = os.path.join(images_dir, image_file)
        dst_image_path = os.path.join(set_images_dir, image_file)
        shutil.copyfile(src_image_path, dst_image_path)
        if verbose: print(f"Copiando imagem '{image_file}'...")

        # Obter as dimensões da imagem
        with Image.open(src_image_path) as img:
            width, height = img.size
            if verbose: print(f"Dimensões da imagem '{image_file}': {width}x{height}")

        # Adicionar entrada para a imagem
        images.append({
            'id': image_id,
            'file_name': image_file,
            'width': width,
            'height': height
        })

        # Ler o arquivo de anotações correspondente
        annotation_file = os.path.splitext(image_file)[0] + '.txt'
        annotation_path = os.path.join(annotation_files, annotation_file)
        if os.path.exists(annotation_path):
            with open(annotation_path, 'r') as f:
                lines = f.readlines()

            for line in lines:
                line = line.strip()
                if not line:
                    continue

                parts = line.split()
                if len(parts) != 4:
                    print(f"Aviso: Formato inválido na linha '{line}' do arquivo '{annotation_path}'.")
                    continue

                try:
                    x_min, y_min, x_max, y_max = map(float, parts)
                except ValueError:
                    print(f"Aviso: Valores não numéricos na linha '{line}' do arquivo '{annotation_path}'.")
                    continue

                # Garantir que as coordenadas estão dentro dos limites da imagem
                x_min = max(0, min(x_min, width - 1))
                y_min = max(0, min(y_min, height - 1))
                x_max = max(0, min(x_max, width - 1))
                y_max = max(0, min(y_max, height - 1))

                bbox_width = x_max - x_min
                bbox_height = y_max - y_min

                if bbox_width <= 0 or bbox_height <= 0:
                    continue

                # Alterar o category_id para 1 (evitar o erro do KeyError)
                annotations.append({
                    'id': annotation_id,
                    'image_id': image_id,
                    'category_id': 1,  # Alterar para 1
                    'bbox': [x_min, y_min, bbox_width, bbox_height],
                    'area': bbox_width * bbox_height,
                    'iscrowd': 0
                })
                annotation_id += 1
        else:
            print(f"Aviso: Arquivo de anotação '{annotation_path}' não encontrado.")

    return images, annotations

# Função para dividir os dados e processar o conjunto de treino, validação e teste
def process_all_sets():
    # Ler a lista de imagens
    set_list_file = os.path.join(img_list_dir, 'train.txt')
    with open(set_list_file, 'r') as f:
        image_files = [line.strip() for line in f.readlines()]

    # Definir a porcentagem para o conjunto de validação (10% do total)
    val_size = int(len(image_files) * 0.1)
    val_image_files = random.sample(image_files, val_size)
    train_image_files = [img for img in image_files if img not in val_image_files]

    # Separar as imagens restantes para o conjunto de teste (caso necessário)
    # Exemplo: 10% para teste, 45% para treino, 45% para validação
    test_size = len(image_files) - val_size - len(train_image_files)
    test_image_files = random.sample([img for img in image_files if img not in train_image_files and img not in val_image_files], test_size)

    # Diretórios para anotações
    train_annotations = annotations_dir
    val_annotations = annotations_dir
    test_annotations = annotations_dir

    # Processar o conjunto de treino
    train_images, train_annotations_data = process_set('train', train_image_files, train_annotations)
    # Processar o conjunto de validação
    val_images, val_annotations_data = process_set('val', val_image_files, val_annotations)
    # Processar o conjunto de teste
    test_images, test_annotations_data = process_set('test', test_image_files, test_annotations)

    # Criar a estrutura final para os arquivos JSON
    coco_format_train = {
        'images': train_images,
        'annotations': train_annotations_data,
        'categories': [
            {
                'id': 1,  # Alterar para 1
                'name': 'Copa de Arvore',
                'supercategory': 'none'
            }
        ]
    }

    coco_format_val = {
        'images': val_images,
        'annotations': val_annotations_data,
        'categories': [
            {
                'id': 1,  # Alterar para 1
                'name': 'Copa de Arvore',
                'supercategory': 'none'
            }
        ]
    }

    coco_format_test = {
        'images': test_images,
        'annotations': test_annotations_data,
        'categories': [
            {
                'id': 1,  # Alterar para 1
                'name': 'Copa de Arvore',
                'supercategory': 'none'
            }
        ]
    }

    # Salvar os arquivos JSON para treino, validação e teste
    with open(os.path.join(output_annotations_dir, 'instances_train.json'), 'w') as f:
        json.dump(coco_format_train, f, indent=4)

    with open(os.path.join(output_annotations_dir, 'instances_val.json'), 'w') as f:
        json.dump(coco_format_val, f, indent=4)

    with open(os.path.join(output_annotations_dir, 'instances_test.json'), 'w') as f:
        json.dump(coco_format_test, f, indent=4)

    print(f"Processamento completo! Arquivos 'instances_train.json', 'instances_val.json' e 'instances_test.json' foram gerados.")

# Chamar a função para processar todos os conjuntos
process_all_sets()


Copiando imagem '100.png'...
Dimensões da imagem '100.png': 512x512
Copiando imagem '35.png'...
Dimensões da imagem '35.png': 512x512
Copiando imagem '125.png'...
Dimensões da imagem '125.png': 512x512
Copiando imagem '19.png'...
Dimensões da imagem '19.png': 512x512
Copiando imagem '40.png'...
Dimensões da imagem '40.png': 512x512
Copiando imagem '122.png'...
Dimensões da imagem '122.png': 512x512
Copiando imagem '89.png'...
Dimensões da imagem '89.png': 512x512
Copiando imagem '22.png'...
Dimensões da imagem '22.png': 512x512
Copiando imagem '93.png'...
Dimensões da imagem '93.png': 512x512
Copiando imagem '43.png'...
Dimensões da imagem '43.png': 512x512
Copiando imagem '1.png'...
Dimensões da imagem '1.png': 512x512
Copiando imagem '103.png'...
Dimensões da imagem '103.png': 512x512
Copiando imagem '130.png'...
Dimensões da imagem '130.png': 512x512
Copiando imagem '42.png'...
Dimensões da imagem '42.png': 512x512
Copiando imagem '49.png'...
Dimensões da imagem '49.png': 512x512
Co

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
!python3 /content/RT-DETR/rtdetr_pytorch/tools/train.py \
-c /content/RT-DETR/rtdetr_pytorch/configs/rtdetr/rtdetr_r50vd_6x_coco.yml


2024-11-30 01:49:34.694011: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-30 01:49:34.713652: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 01:49:34.734870: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 01:49:34.741391: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 01:49:34.756753: I tensorflow/core/platform/cpu_feature_guar

In [15]:
!for epoch in range(epochs):
    # seu código de treinamento aqui


/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `for epoch in range(epochs):'


In [ ]:
!rm -rf ./output  # Exclua os arquivos de saída de treinamentos anteriores
!rm -rf ~/.cache  # Limpe o cache do PyTorch

In [ ]:
import os
print(os.listdir('/content/individual_urban_tree_crown_detection/annotations/'))

['instances_test.json', 'instances_train.json', 'instances_val.json']
